In [1]:
import pandas as pd
import statsmodels.api as sm
# import statsmodels.api as sm2
import matplotlib.pyplot as plt;
import numpy as np
import copy
from collections import defaultdict
from sklearn.model_selection import KFold
df = pd.read_csv("D:/class/cogs109/project/data.csv");

#return: the MSE
#param: y-n*1 list, X-n*m list, coef- 1*m list
# y is the result, x is features, coef is the coeficiency
def find_mse(y,X , coef):
    ##need to do
    total = 0
    for i in range(len(X)):
        total += (sum(X[i] * coef) - y[i]) ** 2
    return total / len(y);

#########################model selection################

###stepwise forwrd part with logistic regression

# feature column No 2 - 31
#stepwise to find the params
valid_column = [x for x in range(2, 32)];
valid_x_index = [];    #save new valid feature's col number in data set
patten = []
alpha = 0.05;
for i in range(len(df)):
    patten.append([1]);
mse_flag = 1;    #mse flag control whether jump out of the while loop
while True:
    coef = defaultdict(list);
    y = [1 if d == 'M' else 0 for d in df['diagnosis']]  #set m is 1 and b is 0
    for i in valid_column:
        x = copy.deepcopy(patten);

        for j in range(len(df.iloc[:,i])):
            x[j].append(df.iloc[j, i]);        #add new featrue value to the end of the previous  [1, x1, x2 ,,,,,]

        #build model and find parameter
        ret = sm.OLS(y, x).fit();
        coef[i] = ret.params;
        #check the any feature is not significant(pvalue > alpha)
        feature_indexof_model = len(valid_x_index) + 1;
        for j in range(1, feature_indexof_model + 1):
            if ret.pvalues[j] > alpha:

                #set the coef of constant to 2 and other coef is 0, make the mse value always are greater than 1
                coef[i][0] = 2;
                for k in range(1, len(coef[i])):
                    coef[i][k] = 0;
                break;



    mse = []        #save this loop all possible mse  (mse value, index of feature)
    for i in valid_column:
        x = copy.deepcopy(patten);
        for j in range(len(df.iloc[:,i])):
            x[j].append(df.iloc[j,i]);
        temp_mse = find_mse(y, x, coef[i]);   #find index is i's value add to previous x to find the new mse
        mse.append((temp_mse, i));

    #check the mse is it decrease
    if min(mse)[0] < mse_flag:
        mse_flag = min(mse)[0];
        x_valid = min(mse)[1];      #find index of the featrue it is most importan
        valid_column.remove(x_valid);  #remove the index from check index list
        valid_x_index.append(x_valid);    #collect the index for feature calculate
        for i in range(len(patten)):
            patten[i].append(df.iloc[i, x_valid]);
    else:
        break;
    if len(valid_column) == 0:    #have add every feature to model
        break;

print("userful feature (orderd) is ", valid_x_index);
print(mse_flag)


#k-fold CV  #############################

#first step use k-fold CV test all different models with lasso and ridge regularization

k = 10
test_mse = []   #save each model's mes    (mse value, model size);

x_index_range = []  #collect the features indexs in the range
for index in range(len(valid_x_index)):
    x_index_range.append(valid_x_index[index]);
    X = np.array(df.iloc[:, x_index_range])
    y = np.array(y);

    kf = KFold(n_splits = k);
    kfold_mse = [];
    for train_index, test_index in kf.split(X):
        a = X[test_index];
        b = y[test_index]
        a = sm.add_constant(a);


###compare ridge and lasso regression  ridge is more accurater than lasso
        # ret = sm.OLS(b, a).fit_regularized(method='elastic_net', L1_wt=1);
        ret = sm.OLS(b, a).fit_regularized(method='elastic_net', L1_wt=0);
        kfold_mse.append(find_mse(b, a, ret.params))
    test_mse.append((np.mean(kfold_mse), index + 1));




best_mse = min(test_mse);
print("best_mse is ", best_mse)
mode_feature_size = test_mse.index(best_mse) + 1;
best_model = valid_x_index[:mode_feature_size]     #final find best model by kfold
print("useful feature column number is (by order)", best_model)



######################Model fit/parameter estimation############################

ret = sm.OLS(y,sm.add_constant(df.iloc[:,best_model  ])).fit();
beta = ret.params;
print("coef is : ", beta);

#######################Estimate parameter SE##########################
####bootstrap to find the confidence interval
n = 100   #resample times
slops = [];    #save all slop for the 100(sample) * 12(features)
for i in range(len(best_model) + 1):
    slops.append([]);
for i in range(n):
    bs = df.sample(len(df), replace = True);       #bootstrap new data
    y = [1 if d == 'M' else 0 for d in bs['diagnosis']]
    ret = sm.OLS(y,sm.add_constant(bs.iloc[:,best_model  ])).fit();
    for j in range(len(best_model) + 1):   #params total has best_model + 1's slop (const);
        slops[j].append(ret.params[j])

#same feature's mean slop_mean
slop_mean = [0] * (len(best_model) + 1);
#same feature's SD
sd = [0] * (len(best_model) + 1);
for i in range(len(slop_mean)):
    slop_mean[i] = np.mean(slops[i]);
    sd[i] = np.std(slops[i]);

#calculate slop SE
CI = [];
for i in range(1, len(slop_mean)):
    CI.append((slop_mean[i] - 2*sd[i]/np.sqrt(100), slop_mean[i] + 2*sd[i]/np.sqrt(100)))
# lower_bar = 0;
# higher_bar = 0;
# for d in CI:
#     lower_bar += d[0];
#     higher_bar += d[1];

print("confidence interval:")
print(CI)


FileNotFoundError: File b'D:/class/cogs109/project/data.csv' does not exist